In [ ]:
#datasat
import networkx as nx
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader
import torch

def Read_graph(file_name):
    edge = np.loadtxt(file_name).astype(np.int32)
    min_node, max_node = edge.min(), edge.max()
    if min_node == 0:
        Node = max_node + 1
    else:
        Node = max_node
    # Node = 504
    G = nx.Graph()
    Adj = np.zeros([Node, Node], dtype=np.int32)
    for i in range(edge.shape[0]):
        G.add_edge(edge[i][0], edge[i][1])
        if min_node == 0:
            Adj[edge[i][0], edge[i][1]] = 1
            Adj[edge[i][1], edge[i][0]] = 1
        else:
            Adj[edge[i][0] - 1, edge[i][1] - 1] = 1
            Adj[edge[i][1] - 1, edge[i][0] - 1] = 1
    Adj = torch.FloatTensor(Adj)
    return G, Adj, Node

class Dataload(data.Dataset):

    def __init__(self, Adj, Node):
        self.Adj = Adj
        self.Node = Node
    def __getitem__(self, index):
        return index
        # adj_batch = self.Adj[index]
        # adj_mat = adj_batch[index]
        # b_mat = torch.ones_like(adj_batch)
        # b_mat[adj_batch != 0] = self.Beta
        # return adj_batch, adj_mat, b_mat
    def __len__(self):
        return self.Node

In [ ]:
#create coefficient matrix
import numpy as np
import pandas as pd 

medicine_name_order = pd.read_table("../input/yaomingzi/entity2id.txt",header=None,encoding='gbk',index_col=0)
medicine_name_order = list(medicine_name_order.index)[0:480]

#row:medicine  column:compond
medi2comp = pd.read_csv("../input/womendeai/zhongyao_hhw_matrix_zuizhong_all.csv",index_col=0)
medi2comp = medi2comp.fillna(value=0)
medi2comp = medi2comp.loc[medicine_name_order,:]
#print(medi2comp.shape) (480,12735)
#row:compond column:protein
comp2prot = pd.read_csv("../input/womendeai/myda.csv",index_col=0)
comp2prot = comp2prot.fillna(value=0)
comp_names = list(medi2comp.columns)
comp2prot = comp2prot.loc[comp_names,:]
#print(comp2prot.shape) (12735,24)
def normalized(x):
    if np.max(x)==0:
        x == 0
    else:
        x -= np.min(x) #为了稳定地计算softmax概率， 一般会减掉最大的那个元素
        x = x / (np.max(x)-np.min(x))
    return x

#compond absorb rate
comp_caco2 = pd.read_csv("../input/hhwcaco2/zhongyao_hhw_CACO2.csv",encoding='gbk')
comp_caco2 = comp_caco2.loc[:,["name","MOL_ID","CACO2"]]
comp_caco2 = comp_caco2.fillna(value=0)
comp_caco2.head
comp_caco2.loc[:,"CACO2"] = comp_caco2.loc[:,"CACO2"] - min(comp_caco2.loc[:,"CACO2"])
comp_caco2.loc[:,"CACO2"] = comp_caco2.loc[:,"CACO2"]/(max(comp_caco2.loc[:,"CACO2"])-min(comp_caco2.loc[:,"CACO2"]))
#print(comp_caco2.shape)
#缺失值归一化后为0.780952
comp_caco2 = comp_caco2.drop_duplicates(["name","MOL_ID"],keep='first')
#print(comp_caco2.shape)
select_medi = medi2comp.iloc[0]
cand_comp = comp_caco2[comp_caco2["name"]==select_medi.name]
cand_comp = cand_comp.sort_values(by='MOL_ID')
#print(cand_comp)
comp_absorb_rate = np.array(cand_comp.loc[:,"CACO2"]).reshape(-1,1)
#comp_absorb_rate
select_comp = select_medi[select_medi==1.0].index.tolist()
medi_combine_prot = comp2prot.loc[select_comp,]
medi_combine_prot = medi_combine_prot.sort_values(by='name')
medi_combine_prot = np.array(medi_combine_prot)
medi_combine_prot = comp_absorb_rate * medi_combine_prot
medi_combine_prot = -np.sum(medi_combine_prot,axis=0).reshape((1,-1))
medi_combine_prot = normalized(medi_combine_prot)
#medi_combine_prot

In [ ]:
for i in range(1,medi2comp.shape[0]):
    #print(i)
    select_medi = medi2comp.iloc[i]
    cand_comp = comp_caco2[comp_caco2["name"]==select_medi.name]
    cand_comp = cand_comp.sort_values(by='MOL_ID')
    select_comp = select_medi[select_medi==1.0].index.tolist()
    cand_comp = comp_caco2[comp_caco2["name"]==select_medi.name]
    if len(select_comp) != 0:
        candicate = comp2prot.loc[select_comp,]
        candicate = candicate.sort_values(by='name')
        if candicate.shape[0] != cand_comp.shape[0]:
            cand_comp = cand_comp[cand_comp["MOL_ID"].isin(list(candicate.index))]
            comp_absorb_rate = np.array(cand_comp.loc[:,"CACO2"]).reshape(-1,1)
            candicate = np.array(candicate)
            candicate = comp_absorb_rate * candicate
            candicate = -np.sum(candicate,axis=0).reshape((1,-1))
            candicate = normalized(candicate)
            medi_combine_prot = np.concatenate((medi_combine_prot,candicate),axis=0)    
        else:
            comp_absorb_rate = np.array(cand_comp.loc[:,"CACO2"]).reshape(-1,1)
            candicate = np.array(candicate)
            candicate = comp_absorb_rate * candicate
            candicate = -np.sum(candicate,axis=0).reshape((1,-1))
            candicate = normalized(candicate)
            medi_combine_prot = np.concatenate((medi_combine_prot,candicate),axis=0)
    else:
        candicate = np.zeros(24)
        medi_combine_prot = np.concatenate((medi_combine_prot,candicate),axis=0)
print(medi_combine_prot.shape)

In [ ]:
medi_combine_prot[:,0] = 20*medi_combine_prot[:,0]
medi_combine_prot[:,1:3] = 10*medi_combine_prot[:,1:3]
medi_combine_prot[:,3] = 15*medi_combine_prot[:,3]
print(medi_combine_prot.shape)
medi_combine_prot_T = medi_combine_prot.T


In [ ]:
print(medi_combine_prot.shape)
#medi_combine_prot[:,4:] = 1*medi_combine_prot[:,4:]

In [ ]:
#set:medicine-A;compond-B;protein-C
A2B = np.array(medi2comp)
A2B_rowsum = np.sum(A2B,axis=1)
A2B_prob = A2B/A2B_rowsum[:,None]
A2B_prob = np.nan_to_num(A2B_prob)

B2A = np.transpose(A2B)
B2A_rowsum = np.sum(B2A,axis=1)
B2A_prob = B2A/B2A_rowsum[:,None]
B2A_prob = np.nan_to_num(B2A_prob)

B2C = np.array(comp2prot)
B2C[B2C>-7] = 0
B2C[B2C<=-7] = 1
B2C_rowsum = np.sum(B2C,axis=1)
B2C_prob = B2C/B2C_rowsum[:,None]
B2C_prob = np.nan_to_num(B2C_prob)

C2B = np.transpose(B2C)
C2B_rowsum = np.sum(C2B,axis=1)
C2B_prob = C2B/C2B_rowsum[:,None]
C2B_prob = np.nan_to_num(C2B_prob)

#result matrix sets M
M11 = np.dot(A2B_prob,B2A_prob)
M22 = np.dot(C2B_prob,B2C_prob)
M12 = np.dot(A2B_prob,B2C_prob)
M21 = np.dot(C2B_prob,B2A_prob)
M1 = np.concatenate((M11,M12),axis=1)
M2 = np.concatenate((M21,M22),axis=1)
M = np.concatenate((M1,M2),axis=0)
#coefficient_matrix create graph

from_list = []
to_list = []
for i in range(M.shape[0]):
    for j in range(M.shape[0]):
        if i != j :
            if M[i,j] > 0 :
                from_list.append(i+1)
                to_list.append(j+1)
adj_biaohhh = np.array([from_list,to_list]).T
adj_biaohhh = adj_biaohhh.astype('int32')
np.savetxt("./biaobiao.txt",adj_biaohhh,delimiter=" ")


In [ ]:
#coefficient matrix
aaa = np.concatenate((M11,medi_combine_prot),axis=1)
bbb = np.concatenate((medi_combine_prot_T,M22),axis=1)
coefficient_matrix = np.concatenate((aaa,bbb),axis=0)
#coefficient_matrix = pd.DataFrame(coefficient_matrix)
#coefficient_matrix.to_csv("./coefficient_matrix.csv")
coefficient_matrix = torch.Tensor(coefficient_matrix)

In [ ]:
coefficient_matrix.shape

In [ ]:
#models
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class MNN(nn.Module):
    def __init__(self, node_size, nhid0, nhid1, droput, alpha):
        super(MNN, self).__init__()
        self.encode0 = nn.Linear(node_size, nhid0)
        self.encode1 = nn.Linear(nhid0, nhid1)
        self.decode0 = nn.Linear(nhid1, nhid0)
        self.decode1 = nn.Linear(nhid0, node_size)
        self.droput = droput
        self.alpha = alpha

    def forward(self, adj_batch, adj_mat, b_mat):
        t0 = F.leaky_relu(self.encode0(adj_batch))
        t0 = F.leaky_relu(self.encode1(t0))
        embedding = t0
        t0 = F.leaky_relu(self.decode0(t0))
        t0 = F.leaky_relu(self.decode1(t0))
        embedding_norm = torch.sum(embedding * embedding, dim=1, keepdim=True)
        L_1st = torch.sum(adj_mat * (embedding_norm -
                                     2 * torch.mm(embedding, torch.transpose(embedding, dim0=0, dim1=1))
                                     + torch.transpose(embedding_norm, dim0=0, dim1=1)))
        L_2nd = torch.sum(((adj_batch - t0) * b_mat) * ((adj_batch - t0) * b_mat))
        return L_1st, self.alpha * L_2nd, L_1st + self.alpha * L_2nd

    def savector(self, adj):
        t0 = self.encode0(adj)
        t0 = self.encode1(t0)
        return t0

In [ ]:
import torch
import torch.optim as optim
import argparse
from torch.utils.data.dataloader import DataLoader
import numpy as np

import networkx as nx
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader
import torch,csv
import pandas as pd
seed = 0
torch.manual_seed(seed) # 为CPU设置随机种子

def parse_args():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                            conflict_handler='resolve')
    parser.add_argument('--input', default='../input/jiuming/biaomatrix.txt',
                        help='Input graph file')
    parser.add_argument('--output', default='./vec.txt',
                        help='Output representation file')
    parser.add_argument('--workers', default=8, type=int,
                        help='Number of parallel processes.')
    parser.add_argument('--weighted', action='store_true', default=False,
                        help='Treat graph as weighted')
    parser.add_argument('--epochs', default=1000, type=int,
                        help='The training epochs of SDNE')
    parser.add_argument('--dropout', default=0.5, type=float,
                        help='Dropout rate (1 - keep probability)')
    parser.add_argument('--weight-decay', type=float, default=5e-4,
                        help='Weight for L2 loss on embedding matrix')
    parser.add_argument('--lr', default=0.001, type=float,
                        help='learning rate')
    parser.add_argument('--alpha', default=1, type=float,
                        help='alhpa is a hyperparameter in SDNE')
    parser.add_argument('--beta', default=5., type=float,
                        help='beta is a hyperparameter in SDNE')
    parser.add_argument('--nu1', default=1e-5, type=float,
                        help='nu1 is a hyperparameter in SDNE')
    parser.add_argument('--nu2', default=1e-4, type=float,
                        help='nu2 is a hyperparameter in SDNE')
    parser.add_argument('--bs', default=100, type=int,
                        help='batch size of SDNE')
    parser.add_argument('--nhid0', default=1000, type=int,
                        help='The first dim')
    parser.add_argument('--nhid1', default=128, type=int,
                        help='The second dim')
    parser.add_argument('--step_size', default=10, type=int,
                        help='The step size for lr')
    parser.add_argument('--gamma', default=0.9, type=int,
                        help='The gamma for lr')
    args = parser.parse_args(args=[])

    return args

In [ ]:
args = parse_args()
G, Adj, Node =  Read_graph(args.input)
# 自身数据集 石膏这个节点与任何药物和蛋白没有任何关联
model = MNN(Node, args.nhid0, args.nhid1, args.dropout, args.alpha)
opt = optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=args.step_size, gamma=args.gamma)
Data = Dataload(Adj, Node)
Data = DataLoader(Data, batch_size=args.bs, shuffle=True, )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
model.train()
for epoch in range(1, args.epochs + 1):
    loss_sum, loss_L1, loss_L2, loss_reg = 0, 0, 0, 0
    for index in Data:
        #adj_batch = Adj[index]
        adj_batch = coefficient_matrix[index]
        #adj_mat = adj_batch[:, index]
        adj_mat = adj_batch[:, index]
        #adj_batch = Adj[index]
        
        #b_mat = torch.ones_like(adj_batch)
        b_mat = coefficient_matrix[index]
        #b_mat[adj_batch != 0] = args.beta

        opt.zero_grad()
        L_1st, L_2nd, L_all = model(adj_batch, adj_mat, b_mat)
        L_reg = 0
        for param in model.parameters():
            L_reg += args.nu1 * torch.sum(torch.abs(param)) + args.nu2 * torch.sum(param * param)
        Loss = L_all + L_reg
        Loss.backward()
        opt.step()
        loss_sum += Loss
        loss_L1 += L_1st
        loss_L2 += L_2nd
        loss_reg += L_reg
    scheduler.step(epoch)
    # print("The lr for epoch %d is %f" %(epoch, scheduler.get_lr()[0]))
    print("loss for epoch %d is:" %epoch)
    print("loss_sum is %f" %loss_sum)
    #print("loss_L1 is %f" %loss_L1)
    #print("loss_L2 is %f" %loss_L2)
    #print("loss_reg is %f" %loss_reg)
model.eval()
embedding = model.savector(Adj)
outVec = embedding.detach().numpy()
np.savetxt(args.output, outVec)

In [ ]:
import pandas as pd
import numpy as np
import os
import csv

# mapping
entity2id = {}
id2entity = {}
with open(r"../input/yaomingzi/entity2id.txt", newline='', encoding='gbk') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['entity', 'id'])
    for row_val in reader:
        id = row_val['id']
        entity = row_val['entity']

        entity2id[entity] = int(id)
        id2entity[int(id)] = entity

# print("Number of entities: {}".format(len(entity2id)))
entity_emb = outVec

In [ ]:
# General Entity Embedding Clustering

from matplotlib import cm
import matplotlib.pyplot as plt
from sklearn.utils import check_random_state

dataset_id = {}
with open( r"../input/yaomingzi/entity2id.txt", newline='', encoding='gbk') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['entity','id'])
    for row_val in reader:
        id = int(row_val['id'])
        #print(id)

        if id <= 479:
            entity_key = "drug"
            if dataset_id.get(entity_key, None) is None:
                dataset_id[entity_key] = []
            dataset_id[entity_key].append(row_val['id'])
        else:
            entity_key = "protein"
            if dataset_id.get(entity_key, None) is None:
                dataset_id[entity_key] = []
            dataset_id[entity_key].append(row_val['id'])
# Calculate entity cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(entity_emb)
#print(similarity.shape)
protein_id = {}
for key in dataset_id["protein"]:
    protein_key = key
    if protein_id.get(protein_key, None) is None:
        protein_id[protein_key] = []
    protein_id[protein_key].append(id2entity[int(key)])
print(protein_id)

In [ ]:
cluster_id = eval(input("请输入需要聚类的蛋白的id(可多个，逗号分开,类似这种[480])："))
#print(cluster_id)
cossim_total = []
for i in range(480):
    for j in cluster_id:
        cossim = []
        cossim.append(similarity[i][j])
        #print(similarity[i][j])
    cossim_total.append(cossim)

cossim_ave = []
for sim in cossim_total:
    sim_ave = np.mean(sim)
    cossim_ave.append(sim_ave)
cossim_ave = np.array(cossim_ave)
cossim_ave_sort = -(np.sort(-cossim_ave))
print(cossim_ave_sort[:10])
cossim_ave_sort_indx = np.argsort(-cossim_ave)
print(cossim_ave_sort_indx[:20])
for i in cossim_ave_sort_indx[:20]:
    print(id2entity[i])